In [1]:
import numpy as np
import shorttext

//anaconda/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# loading the whole ESV Bible
import biblebooks.bibledocs_iterator as bit
dbconn = bit.get_sqlite3_dbconn("ESVCrossway.sqlite")
biblecorpus = bit.generate_classdict_chapters(dbconn)

In [3]:
from biblebooks import BibleAbbrDict

In [4]:
# loading FastText model
wvmodel = shorttext.utils.load_fasttext_model('/Users/hok/Data/FastText/wiki.simple/wiki.simple.bin')

In [5]:
# define neural network

In [6]:
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, Activation, Input
from keras.models import Sequential, Model
from keras.regularizers import l2

In [7]:
maxlen = 20

input_x = Input(shape=(maxlen, wvmodel.vector_size))
output_y = LSTM(1024, return_sequences=True)(input_x)
print(output_y.shape)
output_y = Conv1D(filters=1200, kernel_size=2)(output_y)
print(output_y.shape)
output_y = MaxPooling1D(pool_size=19)(output_y)
print(output_y.shape)
output_y = Flatten()(output_y)
print(output_y.shape)
output_y = Dense(2, activation='sigmoid')(output_y)
print(output_y.shape)

kmodel = Model(inputs=input_x, outputs=output_y)

(?, ?, 1024)
(?, ?, 1200)
(?, ?, 1200)
(?, ?)
(?, 2)


In [8]:
kmodel.compile(loss='categorical_crossentropy', optimizer='adam')

In [9]:
maxlen = 20

smodel = Sequential()
smodel.add(LSTM(1024, return_sequences=True, input_shape=(maxlen, wvmodel.vector_size)))
smodel.add(Conv1D(filters=1200, kernel_size=2))
smodel.add(MaxPooling1D(pool_size=maxlen-2+1))
smodel.add(Flatten())
smodel.add(Dense(2, activation='sigmoid'))

smodel.compile(loss='categorical_crossentropy', optimizer='adam')

In [10]:
newdict = {}
newdict['ot'] = [sent for otbookabbr in BibleAbbrDict.otbookdict  if otbookabbr=='de'
                 for chap in range(1, BibleAbbrDict.numchaps[otbookabbr]+1) 
                 for sent in biblecorpus[otbookabbr+'_'+str(chap)]]
newdict['nt'] = [sent for ntbookabbr in BibleAbbrDict.ntbookdict if ntbookabbr=='mt'
                 for chap in range(1, BibleAbbrDict.numchaps[otbookabbr]+1) 
                 for sent in biblecorpus[otbookabbr+'_'+str(chap)]]

In [11]:
classifier = shorttext.classifiers.VarNNEmbeddedVecClassifier(wvmodel, maxlen=maxlen)

In [12]:
classifier.train(newdict, kmodel)

Epoch 1/10
1297/1297 [==============================] - 23s 17ms/step - loss: 0.5670
Epoch 2/10
1297/1297 [==============================] - 22s 17ms/step - loss: 0.5054
Epoch 3/10
1297/1297 [==============================] - 22s 17ms/step - loss: 0.5052
Epoch 4/10
1297/1297 [==============================] - 21s 16ms/step - loss: 0.5089
Epoch 5/10
1297/1297 [==============================] - 21s 16ms/step - loss: 0.5082
Epoch 6/10
1297/1297 [==============================] - 25s 19ms/step - loss: 0.5062
Epoch 7/10
1297/1297 [==============================] - 22s 17ms/step - loss: 0.5059
Epoch 8/10
1297/1297 [==============================] - 21s 16ms/step - loss: 0.5046
Epoch 9/10
1297/1297 [==============================] - 21s 16ms/step - loss: 0.5055
Epoch 10/10
1297/1297 [==============================] - 21s 16ms/step - loss: 0.5038


In [13]:
classifier.score('Love the Lord with all your heart.')

{'nt': 0.3086999, 'ot': 0.9113841}

In [14]:
classifier.score('Pray in this way as I have instructed you.')

{'nt': 0.28103402, 'ot': 0.9383499}

In [15]:
classifier.score('Moses speaks.')

{'nt': 0.25501776, 'ot': 0.9709943}

In [16]:
classifier.score('Jesus Christ')

{'nt': 0.2820653, 'ot': 0.9417404}

In [17]:
classifier.score('False idols and true son of God.')

{'nt': 0.26877484, 'ot': 0.97342694}

In [18]:
classifier.score('Abraham died.')

{'nt': 0.2601503, 'ot': 0.97206926}

In [19]:
classifier.save_compact_model('de_mt_model_v2.bin')